# Finding Hierarchical Ingredient Code List codes

Most medications in eICU have a HICL code which we can use to find them in the database.

## 0. Setup

In [1]:
# Must install pandas-gbq. Link: https://pandas-gbq.readthedocs.io/en/latest/install.html#pip
import math
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

project_id='lcp-internal'

# Helper function to read data from BigQuery into pandas dataframes.
def run_query(query):
    return pd.io.gbq.read_gbq(query,
                              project_id=project_id, verbose=False,
                              dialect='standard')

# 1. Find data which has that drug name

This is a crude search that aims to (1) find a few HICLs and (2) find synonyms to the drug name (e.g. brand names).

In [2]:
# cohort with exclusions applied
compound = 'lasix'
query = f"""
select
    drughiclseqno, drugname, count(*) as n
from `physionet-data.eicu_crd.medication` med
where lower(drugname) like '%{compound}%'
group by drugname, drughiclseqno
order by n desc
"""
med = run_query(query)

print(med.shape)
display(med)

(17, 3)


,drughiclseqno,drugname,n
0,3660,LASIX,17801
1,3660,Lasix,9301
2,3660,furosemide (LASIX) inj 40 mg,3485
3,3660,furosemide (LASIX) injection 40 mg,1679
4,3660,furosemide (LASIX) inj 20 mg,1530
5,3660,furosemide (LASIX) tablet 40 mg,1212
6,3660,furosemide (LASIX) tab 40 mg,1078
7,3660,furosemide (LASIX) 10 mg/mL injection 40 mg,1033
8,3660,furosemide (LASIX) injection 20 mg,1005
9,None,LASIX,521


From the above, we can be fairly confident that 3660 is our HICL code. Of course, it's also worth noting that we need to retain a search for the drug name as not all rows have a HICL code.

We now proceed to expand our search to (1) the new HICL and (2) any other names which may be associated with lasix (e.g. from the above, furosemide is a clear candidate).

In [3]:
# cohort with exclusions applied
hicl = 3660
query = f"""
select
    drughiclseqno, drugname, count(*) as n
from `physionet-data.eicu_crd.medication` med
where drughiclseqno = {hicl}
group by drugname, drughiclseqno
order by n desc
"""
med = run_query(query)
print(med.shape)
display(med)

(35, 3)


,drughiclseqno,drugname,n
0,3660,FUROSEMIDE,29030
1,3660,None,19522
2,3660,LASIX,17801
3,3660,Lasix,9301
4,3660,4 ML - FUROSEMIDE 10 MG/ML IJ SOLN,7797
5,3660,furosemide,6862
6,3660,FUROSEMIDE 10 MG/1 ML 2ML SDV INJ,6862
7,3660,FUROSEMIDE 10 MG/1 ML 4ML SDV INJ,6609
8,3660,FUROSEMIDE 40 MG PO TABS,6202
9,3660,4 ML VIAL : FUROSEMIDE 10 MG/ML IJ SOLN,4811


Looks like the only additional term we should search for is furosemide.

In [4]:
compound1 = 'lasix'
compound2 = 'furosemide'
query = f"""
select
    drughiclseqno, drugname, count(*) as n
from `physionet-data.eicu_crd.medication` med
where lower(drugname) like '%{compound1}%'
OR lower(drugname) like '%{compound2}%'
group by drugname, drughiclseqno
order by n desc
"""
med = run_query(query)
print(med.shape)
display(med)

(61, 3)


,drughiclseqno,drugname,n
0,3660,FUROSEMIDE,29030
1,3660,LASIX,17801
2,3660,Lasix,9301
3,3660,4 ML - FUROSEMIDE 10 MG/ML IJ SOLN,7797
4,3660,FUROSEMIDE 10 MG/1 ML 2ML SDV INJ,6862
5,3660,furosemide,6862
6,3660,FUROSEMIDE 10 MG/1 ML 4ML SDV INJ,6609
7,3660,FUROSEMIDE 40 MG PO TABS,6202
8,3660,4 ML VIAL : FUROSEMIDE 10 MG/ML IJ SOLN,4811
9,3660,2 ML VIAL: FUROSEMIDE 10 MG/ML IJ SOLN,3848


More hits, but not many more HICL codes:

In [5]:
med['drughiclseqno'].value_counts()

3660    34
2854     1
Name: drughiclseqno, dtype: int64

Some codes are used for multiple drugs, so let's double check that 2854 is OK to include.

In [6]:
# cohort with exclusions applied
hicl = 2854
query = f"""
select
    drughiclseqno, drugname, count(*) as n
from `physionet-data.eicu_crd.medication` med
where drughiclseqno = {hicl}
group by drugname, drughiclseqno
order by n desc
"""
med = run_query(query)
print(med.shape)
display(med)


(28, 3)


,drughiclseqno,drugname,n
0,2854,None,216
1,2854,NovoLOG Flexpen,10
2,2854,oxyCODONE-acetaminophen,7
3,2854,methylPREDNISolone,5
4,2854,LABETALOL HCL,5
5,2854,azithromycin,4
6,2854,CLOPIDOGREL,4
7,2854,PANTOPRAZOLE SODIUM,3
8,2854,FUROSEMIDE,3
9,2854,predniSONE,3


From the above, it's clear we don't want to include 2854 as it corresponds to many different drugs.

Consequently, our final search will look for:

* drughiclseqno = 3660
* lower(drugname) like '%lasix%'
* lower(drugname) like '%furosemide%'

```
WHERE drughiclseqno = 3660
   OR lower(drugname) like '%lasix%'
   OR lower(drugname) like '%furosemide%'
```

# Repeat the above for loop diuretics

Repeat the above for other loop diuretics:

* bumex
* torsemide


In [7]:
def hicl_query(hicl):
    query = f"""
    select
        drughiclseqno, drugname, count(*) as n
    from `physionet-data.eicu_crd.medication` med
    where drughiclseqno = {hicl}
    group by drugname, drughiclseqno
    order by n desc
    """
    return run_query(query)

def drug_query(compound, *args):
    where_clause = f"WHERE lower(drugname) like '%{compound}%'"
    if args:
        for a in args:
            where_clause += f" OR lower(drugname) like '%{a}%'"
    query = f"""
    select
        drughiclseqno, drugname, count(*) as n
    from `physionet-data.eicu_crd.medication` med
    {where_clause}
    group by drugname, drughiclseqno
    order by n desc
    """
    return run_query(query)

In [8]:
med = drug_query('bumex', 'bumetanide')
print(med.shape)
display(med)

(9, 3)


,drughiclseqno,drugname,n
0,3664,BUMEX,2962
1,3664,BUMETANIDE,2012
2,3664,BUMETANIDE 0.25 MG/1ML 4ML SDV INJ,1289
3,3664,Bumex,1186
4,None,BUMEX,172
5,None,BUMETANIDE 0.25 MG/1ML 4ML SDV INJ,60
6,None,BUMETANIDE,9
7,18084,BUMEX,2
8,18084,BUMETANIDE,1


In [9]:
med = hicl_query(3664)
display(med)

,drughiclseqno,drugname,n
0,3664,None,6924
1,3664,BUMEX,2962
2,3664,BUMETANIDE,2012
3,3664,BUMETANIDE 0.25 MG/1ML 4ML SDV INJ,1289
4,3664,Bumex,1186


In [10]:
med = hicl_query(18084)
display(med)

,drughiclseqno,drugname,n
0,18084,None,4822
1,18084,HumaLOG,837
2,18084,LORazepam,770
3,18084,PEPCID,294
4,18084,heparin (porcine),282
5,18084,Protonix,185
6,18084,ZOSYN,166
7,18084,ASPIRIN EC,127
8,18084,predniSONE,112
9,18084,Coumadin,86


So for bumex: `WHERE drughiclseqno = 3664 OR lower(drugname) like '%bumex%' OR lower(drugname) like '%bumetanide%'`

### torsemide

In [11]:
med = drug_query('torsemide')
print(med.shape)
display(med)

(0, 3)


,drughiclseqno,drugname,n


In [12]:
med = drug_query('tors')
print(med.shape)
display(med)

(0, 3)


,drughiclseqno,drugname,n


# Repeat for NSAIDs

Also repeat the above for NSAIDs:

* ?Acetaminophen
* Aspirin
* Diflunisal
* Choline magnesium trisalicylate
* Salsalate
* Naproxen
* Ibuprofen
* Ketoprofen
* Flurbiprofen
* Oxaprozin
* Diclofenac
* Etodolac
* Indomethacin
* Tolmetin
* Sulindac
* Meloxicam
* Piroxicam
* Meclofenamate
* Mefenamic acid
* Nabumetone
* toradol/ketorolac

? Selective COX-2 inhibitors

* Celecoxib

Spoiler alert: we only found aspirin/ibuprofen/toradol in the data:

`WHERE drughiclseqno = 1820 OR lower(drugname) like '%aspirin%' OR lower(drugname) like '%ecotrin%'`
`WHERE drughiclseqno = 3723 OR lower(drugname) like '%ibuprofen%' OR lower(drugname) like '%motrin%'`
`WHERE drughiclseqno = 5175 OR lower(drugname) like '%toradol%' OR lower(drugname) like '%ketorolac%'`

## Toradol/Ketorolac

In [13]:
med = drug_query('toradol')
print(med.shape)
display(med)

(2, 3)


,drughiclseqno,drugname,n
0,5175,TORADOL,1875
1,None,TORADOL,95


In [14]:
med = hicl_query(5175)
print(med.shape)
display(med)

(6, 3)


,drughiclseqno,drugname,n
0,5175,None,7486
1,5175,TORADOL,1875
2,5175,KETOROLAC INJ,1705
3,5175,KETOROLAC TROMETHAMINE,1530
4,5175,KETOROLAC 30 MG/ML 1ML SDV INJ,1227
5,5175,KETOROLAC,1224


In [15]:
med = drug_query('toradol', 'ketorolac')
print(med.shape)
display(med)

(11, 3)


,drughiclseqno,drugname,n
0,5175,TORADOL,1875
1,5175,KETOROLAC INJ,1705
2,5175,KETOROLAC TROMETHAMINE,1530
3,5175,KETOROLAC 30 MG/ML 1ML SDV INJ,1227
4,5175,KETOROLAC,1224
5,None,KETOROLAC INJ,216
6,None,TORADOL,95
7,None,KETOROLAC,84
8,None,KETOROLAC 30 MG/ML 1ML SDV INJ,46
9,None,KETOROLAC TROMETHAMINE,6


## Aspirin

In [ ]:
med = drug_query('aspirin')
print(med.shape)
display(med)

med = hicl_query(1820)
print(med.shape)
display(med)

med = drug_query('ecotrin')
print(med.shape)
display(med)

med = hicl_query(1794)
display(med)

med = hicl_query(21996)
display(med)

(45, 3)


,drughiclseqno,drugname,n
0,1820,ASPIRIN 81 MG PO CHEW,9509
1,1820,ASPIRIN,9167
2,1820,aspirin,8447
3,1820,ASPIRIN 325 MG PO TABS,5075
4,1820,ASPIRIN EC 81 MG PO TBEC,4717
5,1820,ASPIRIN (BABY ASA) 81 MG TAB,4151
6,1820,ASPIRIN EC,2950
7,1820,"ASPIRIN, CHILDREN'S",2881
8,1820,ASPIRIN 81 MG EC TAB,2508
9,1820,ASPIRIN 325MG TAB,1920


(25, 3)


,drughiclseqno,drugname,n
0,1820,None,18608
1,1820,ASPIRIN 81 MG PO CHEW,9509
2,1820,ASPIRIN,9167
3,1820,aspirin,8447
4,1820,ASPIRIN 325 MG PO TABS,5075
5,1820,ASPIRIN EC 81 MG PO TBEC,4717
6,1820,ASPIRIN (BABY ASA) 81 MG TAB,4151
7,1820,ECOTRIN EC,3967
8,1820,ASPIRIN EC,2950
9,1820,"ASPIRIN, CHILDREN'S",2881


(4, 3)


,drughiclseqno,drugname,n
0,1820,ECOTRIN EC,3967
1,1820,ECOTRIN,959
2,None,ECOTRIN,97
3,None,ECOTRIN EC,8


,drughiclseqno,drugname,n
0,1794,ASPIRIN,1
1,1794,None,1


## Diflunisal

In [ ]:
med = drug_query('diflun')
print(med.shape)
display(med)

## Choline magnesium trisalicylate

In [ ]:
med = drug_query('choline')
print(med.shape)
display(med)

In [ ]:
med = drug_query('trilisate')
print(med.shape)
display(med)

## Salsalate

In [ ]:
med = drug_query('salsalate')
print(med.shape)
display(med)

## Naproxen

In [ ]:
med = drug_query('naproxen')
print(med.shape)
display(med)

## Ibuprofen

In [ ]:
med = drug_query('ibuprofen')
print(med.shape)
display(med)

In [ ]:
med = hicl_query(3723)
print(med.shape)
display(med)

In [ ]:
med = drug_query('motrin')
print(med.shape)
display(med)

## Ketoprofen

In [ ]:
med = drug_query('ketoprofen')
print(med.shape)
display(med)

## Flurbiprofen

In [ ]:
med = drug_query('flurbiprofen')
print(med.shape)
display(med)

## Oxaprozin

In [ ]:
med = drug_query('oxaprozin')
print(med.shape)
display(med)

## Diclofenac

In [ ]:
med = drug_query('diclofenac')
print(med.shape)
display(med)

## Etodolac

In [ ]:
med = drug_query('etodolac')
print(med.shape)
display(med)

## Indomethacin

In [ ]:
med = drug_query('indomethacin')
print(med.shape)
display(med)

## Tolmetin

In [ ]:
med = drug_query('tolmetin')
print(med.shape)
display(med)

## Sulindac

In [ ]:
med = drug_query('sulindac')
print(med.shape)
display(med)

## Meloxicam

In [ ]:
med = drug_query('meloxicam')
print(med.shape)
display(med)

## Piroxicam

In [ ]:
med = drug_query('piroxicam')
print(med.shape)
display(med)

## Meclofenamate

In [ ]:
med = drug_query('meclofenamate')
print(med.shape)
display(med)

## Mefenamic acid

In [ ]:
med = drug_query('mefenamic')
print(med.shape)
display(med)

## Nabumetone

In [ ]:
med = drug_query('nabumetone')
print(med.shape)
display(med)

In [ ]:
med = drug_query('relafen')
print(med.shape)
display(med)

## Celecoxib (COX-2 inhibitor)

In [ ]:
med = drug_query('celecoxib')
print(med.shape)
display(med)

In [ ]:
med = drug_query('celebrex')
print(med.shape)
display(med)